In [461]:
import os

In [462]:
import warnings
warnings.filterwarnings("ignore") 

In [463]:
from dotenv import load_dotenv
load_dotenv()

True

In [464]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGSMITH_TRACING=os.getenv("LANGSMITH_TRACING_V2", "true")  # Default to "true" if not set
LANGSMITH_ENDPOINT=os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT=os.getenv("LANGSMITH_PROJECT")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING_V2"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

### Load the model and test it with a simple message

In [465]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

In [466]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     convert_system_message_to_human=True,
# )

llm = ChatGroq(model="llama3-8b-8192")

In [467]:
message1 = [SystemMessage(content="You are a helpful assistant."),
           HumanMessage(content="I need your help")]

In [468]:
response1 = llm.invoke(message1)
response1

AIMessage(content="I'd be happy to help you. Please share what's on your mind and what you need help with. Whether it's a question, a problem, or just someone to talk to, I'm here to listen and assist you in any way I can.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 25, 'total_tokens': 78, 'completion_time': 0.06455597, 'prompt_time': 0.004612482, 'queue_time': 0.27770822100000003, 'total_time': 0.069168452}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6da36115-c1ee-42f0-b1fa-639dc06718c2-0', usage_metadata={'input_tokens': 25, 'output_tokens': 53, 'total_tokens': 78})

In [469]:
message2 = [SystemMessage(content="You are a helpful assistant. and you give concise answers."),
           HumanMessage(content="I need your help")]

In [470]:
response2 = llm.invoke(message2)
response2

AIMessage(content="I'd be happy to help you. What's on your mind?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 31, 'total_tokens': 46, 'completion_time': 0.023247517, 'prompt_time': 0.011684492, 'queue_time': 0.439985485, 'total_time': 0.034932009}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7e188399-a4bb-4e05-affe-add35ea3d96d-0', usage_metadata={'input_tokens': 31, 'output_tokens': 15, 'total_tokens': 46})

### Using Output Perser

In [471]:
from langchain_core.output_parsers import StrOutputParser

In [472]:
parser = StrOutputParser()
parser.invoke(response1)

"I'd be happy to help you. Please share what's on your mind and what you need help with. Whether it's a question, a problem, or just someone to talk to, I'm here to listen and assist you in any way I can."

In [473]:
parser.invoke(response2)

"I'd be happy to help you. What's on your mind?"

### Chaining with LCEL
- LangChain Expression Language (LCEL) &mdash; a concise syntax for composing and chaining together components (like models and output parsers) in LangChain workflows.LCEL

In [474]:
chain = llm | parser

- The line `chain = llm | parser` creates a simple pipeline (chain) where the output of the language model (`llm`) is passed directly to the output parser (`parser`). This allows you to process a prompt through the model and automatically parse its response in a single step, making it easier to work with structured outputs in LangChain.

In [475]:
chain.invoke(message1)

"I'd be happy to help you. Please tell me what's on your mind and how I can assist you. Are you facing a problem, looking for information, or just need someone to talk to? I'm all ears!"

In [476]:
chain.invoke(message2)

"I'd be happy to help! What do you need help with?"

### Prompting with PromptTemplate

In [477]:
from langchain_core.prompts import ChatPromptTemplate

In [478]:
promt_template = ChatPromptTemplate.from_messages(
    [("system", "You give concise answer and Translate the following text to : {language}"),
     ("human", "{text}")]
)
    

In [479]:
prompt = promt_template.invoke({"language": "Bengali", "text": "My name is Durjay."})

In [480]:
prompt.to_messages()

[SystemMessage(content='You give concise answer and Translate the following text to : Bengali', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My name is Durjay.', additional_kwargs={}, response_metadata={})]

In [481]:
response = llm.invoke(prompt)

In [482]:
response.content

'My name is দুর্জ্জয় (Durjay).'

### Chaining

In [483]:
chain = promt_template | llm | parser

In [484]:
chain.invoke({"language": "Bengali", "text": "My name is Durjay."})

'আমার নাম দুর্জয়।\n\nTranslation: Amāra nāma durjay.'

### Agent

In [559]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, HumanMessage

In [560]:
memory = MemorySaver()

In [561]:
search = TavilySearchResults(max_results=2)

In [562]:
tools = [search]

In [563]:
agent_executor = create_react_agent(llm,tools, checkpointer=memory)

In [564]:
config = {"configurable": {"thread_id": "abc123"}}

In [565]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im Durjay! and i live in Khulna")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'd1zwdf6cv', 'function': {'arguments': '{"query":"Khulna"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 928, 'total_tokens': 1000, 'completion_time': 0.107112413, 'prompt_time': 0.330904778, 'queue_time': 0.27689512199999994, 'total_time': 0.438017191}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c1b0d236-f11f-4da1-a995-bdfa1e441d63-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Khulna'}, 'id': 'd1zwdf6cv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 928, 'output_tokens': 72, 'total_tokens': 1000})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"title": "Khulna - Wikipedia", "url": "https://en.wikipedia.org/wiki/Khulna", "content": "Khulna (Bengali

In [566]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="Whats my local language?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your local language is Bengali.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 2821, 'total_tokens': 2829, 'completion_time': 0.01055421, 'prompt_time': 0.821808697, 'queue_time': 0.27642795499999995, 'total_time': 0.832362907}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--10e23cbc-b66c-4eae-ad68-bd19e36311d7-0', usage_metadata={'input_tokens': 2821, 'output_tokens': 8, 'total_tokens': 2829})]}}
----
